In [ ]:
import pandas as pd
import glob
import datasets
import numpy as np

In [ ]:
results_dict = {}
results_dict['language'] = []
results_dict['relation'] = []
results_dict['percentage change'] = []
results_dict['new ratio of rows'] = []
results_dict['old ratio of rows'] = []


hf_df = datasets.load_dataset('CalibraGPT/Fact-Completion')
file_names = glob.glob("../../data/result_logs/llama/error-analysis/*.csv")

# confirm grabbing data correctly against LLaMa figure
# uncomment print statement at end of for loop to see
for file in file_names:
    language = file.split('.csv')[0].split('-')[-1].capitalize()
    error_df = pd.read_csv(file)
    full_hf_df = hf_df[file.split('.csv')[0].split('-')[-1].capitalize()]
    # print(language, ":", np.round((len(full_hf_df) - len(error_df))/len(full_hf_df) * 100, 3), "%")

# calculate the distribution shift by language by relation present in the errors
for file in file_names:
    language = file.split('.csv')[0].split('-')[-1].capitalize()
    error_df = pd.read_csv(file)
    full_hf_df = hf_df[file.split('.csv')[0].split('-')[-1].capitalize()]
    full_hf_df = full_hf_df.to_pandas()
    relations = np.unique(np.array(error_df.relation.tolist()))
    for relation in relations:
        new_ratio = len(error_df[error_df["relation"] == relation]) / len(error_df)
        old_ratio = len(full_hf_df[full_hf_df["relation"] == relation]) / len(full_hf_df)
        results_dict['relation'].append(relation)
        results_dict['language'].append(language)
        results_dict['percentage change'].append(np.round(100*(new_ratio - old_ratio)/old_ratio, 3))
        results_dict['new ratio of rows'].append(np.round(new_ratio, 3))
        results_dict['old ratio of rows'].append(np.round(old_ratio, 3))

In [ ]:
df = pd.DataFrame.from_dict(results_dict)
quant_error_analysis = df.sort_values(by=['percentage change'], ascending=False)
quant_error_analysis.to_csv('../../data/error_analysis/canonical_quant_error_analysis.csv')

# top % increase across all languages
quant_error_analysis.head(25)

In [ ]:
# average across all langs

grouped_quant_error_analysis = pd.DataFrame(quant_error_analysis.groupby(['relation'])["percentage change"].mean())

pd_df = {}
pd_df['relation'] = grouped_quant_error_analysis.index.tolist()
pd_df['mean percentage change across langs'] = grouped_quant_error_analysis['percentage change'].tolist()

grouped_quant_error_analysis = pd.DataFrame.from_dict(pd_df)
grouped_quant_error_analysis = grouped_quant_error_analysis.sort_values('mean percentage change across langs', ascending=False).reset_index(drop=True)

grouped_quant_error_analysis.to_csv('../../data/error_analysis/aggregate_quant_error_analysis.csv')
grouped_quant_error_analysis
